In [1]:
import pandas as pd
import datetime as dt
data = pd.read_csv("yt_final.csv", encoding = 'utf8')
data = data[data['cluster'] == 2]
data.head()

,channel_name,video_title,views,comments,duration,likes,publish_date,category,channel_startdate,subscribers,channel_totalviews(10K),Sensationalism_Score,cluster
208,Joeman,高性價比的i9電競筆電！HP OMEN 16 intel i9-13900hx + RTX4...,185000,122,654,935,2023-11-01,People & Blogs,2010-10-11,2590000,101544.6547,40,2
209,Joeman,52500元的奢華棒球場對決200元的平價棒球場！《Joe是要對決S2》Ep184 ft.琳...,526000,1330,1832,8000,2023-10-30,People & Blogs,2010-10-11,2590000,101544.6547,22,2
210,Joeman,台灣無訊號的地方華為手機竟然還能打？Mate 60 Pro衛星通訊實測【Joeman】,409000,3046,737,7700,2023-10-28,People & Blogs,2010-10-11,2590000,101544.6547,19,2
211,Joeman,跟兒子溫馨出遊！帶新豐街底迪出去玩一天！【Joeman】,270000,352,788,5000,2023-10-25,People & Blogs,2010-10-11,2590000,101544.6547,13,2
212,Joeman,可以AI換臉的魔術手機 Pixel 8 Pro開箱【Joeman】,106000,243,912,2100,2023-10-24,People & Blogs,2010-10-11,2590000,101544.6547,25,2


In [2]:
data['publish_date'] = pd.to_datetime(data['publish_date'])
data['channel_startdate'] = pd.to_datetime(data['channel_startdate'])
current_date = pd.Timestamp('now')
data['days_since_publish'] = (current_date - data['publish_date']).dt.days
data['channel_age_days'] = (current_date - data['channel_startdate']).dt.days

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

In [4]:
# 設定特徵及目標變量
features = data.drop(['views','channel_name', 'video_title', 'channel_startdate','cluster','likes', 'comments','publish_date','channel_totalviews(10K)'], axis = 1)
target = data['views']


In [5]:
# Apply One-Hot Encoding to 'category'
one_hot_encoder = OneHotEncoder(sparse=False)
category_encoded = one_hot_encoder.fit_transform(features[['category']])
category_encoded_df = pd.DataFrame(category_encoded, columns=one_hot_encoder.get_feature_names(['category']))

# Drop the original 'category' column and concatenate the one-hot encoded dataframe
features = features.drop('category', axis=1)
features_encoded = pd.concat([features.reset_index(drop=True), category_encoded_df], axis=1)

features_encoded.head()  # Display the first few rows of the processed features

/Users/chenxingchun/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,duration,subscribers,Sensationalism_Score,days_since_publish,channel_age_days,category_Comedy,category_Education,category_Entertainment,category_Music,category_People & Blogs,category_Sports
0,654,2590000,40,20,4789,0.0,0.0,0.0,0.0,1.0,0.0
1,1832,2590000,22,22,4789,0.0,0.0,0.0,0.0,1.0,0.0
2,737,2590000,19,24,4789,0.0,0.0,0.0,0.0,1.0,0.0
3,788,2590000,13,27,4789,0.0,0.0,0.0,0.0,1.0,0.0
4,912,2590000,25,28,4789,0.0,0.0,0.0,0.0,1.0,0.0


In [6]:
X_train, X_test, y_train, y_test = train_test_split(features_encoded, target, test_size=0.2)
# 建立RandomForestRegressor模型
model = GradientBoostingRegressor(random_state=42)
# 定義要搜索的参数分布
param_distributions = {
    'n_estimators': randint(50, 400), 
    'learning_rate': uniform(0.01, 0.2),
    'max_depth': randint(3, 10), 
    'subsample': uniform(0.7, 0.3), 
    'min_samples_split': randint(2, 10) 
} 
# 建立RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_distributions, n_iter=100, cv=5, scoring='r2', n_jobs=-1, random_state=42)
# 執行隨機搜索
random_search.fit(X_train, y_train)

# 最佳參數組合與評分
best_parameters = random_search.best_params_
best_score = random_search.best_score_

print(f"Best Parameters: {best_parameters}")
print(f"Best Score (r2): {best_score}")

Best Parameters: {'learning_rate': 0.07161215837047784, 'max_depth': 6, 'min_samples_split': 4, 'n_estimators': 71, 'subsample': 0.7731968930137251}
Best Score (r2): 0.856975388233159


In [7]:
# 使用這些參數初始化一個新的 GradientBoostingRegressor
optimized_model = GradientBoostingRegressor(
    n_estimators=best_parameters['n_estimators'],
    learning_rate=best_parameters['learning_rate'],
    max_depth=best_parameters['max_depth'],
    subsample=best_parameters['subsample'],
    min_samples_split=best_parameters['min_samples_split'],
    random_state=42
)
# 訓練模型
optimized_model.fit(X_train, y_train)
#進行預測
predicted = optimized_model.predict(X_test)
# 評估模型性能
optimized_score = optimized_model.score(X_test, y_test)
print(f"Optimized Model Score (r2) on Test Data: {optimized_score}")

Optimized Model Score (r2) on Test Data: 0.8815744815069452


### 特徵重要性

In [8]:
import plotly.graph_objects as go
feature_importances = optimized_model.feature_importances_
feature_names = X_train.columns

# 創建特徵重要性的DataFrame
importances_df = pd.DataFrame({'feature': feature_names, 'importance': feature_importances})
importances_df = importances_df.sort_values(by='importance', ascending=False)

# 創建圖表
fig = go.Figure(go.Bar(
    x=importances_df['feature'][:10],
    y=importances_df['importance'][:10],
    text=importances_df['importance'][:10].apply(lambda x: f'{x:.2f}'),
    textposition='outside',
    marker_color='#ffa500',
    width=0.6
))

# 調整布局
fig.update_layout(
    title={
        'text': 'Top 10 Feature Importances in Gradient Boosting Regressor Model',
        'y':0.98,  # 標題在垂直方向的位置
        'x':0.5,  # 標題在水平方向的位置
        'xanchor': 'center',  # 確保標題的中心在 x 的位置
        'yanchor': 'top'      # 確保標題的底部在 y 的位置
    },
    xaxis=dict(
        title='Feature',
        tickangle=-45,
        tickfont=dict(size=14)
    ),
    yaxis=dict(
        title='Importance',
        range=[0, 1.1 * max(importances_df['importance'][:10])]
    ),
    margin=dict(t=50),  # 可能需要根據標題的大小調整邊距
    showlegend=False,
    bargap=0.05,
    width=600, # 調整圖表的寬度
)

# 顯示圖表
fig.show()

### 實際值vs預測值

In [9]:
import plotly.express as px

fig = px.scatter(x=y_test, y=predicted , labels={'x': 'Actual Views', 'y': 'Predicted Views'},
                 title='Actual vs Predicted Views', width=600, color_discrete_sequence=['#ffa500'], opacity=0.7)
fig.add_shape(type='line', 
              x0=y_test.min(), y0=y_test.min(), 
              x1=y_test.max(), y1=y_test.max(),
              line=dict(color='gray', dash='dash'))
fig.update_layout(
    title={
        'y':0.85,  # 標題在垂直方向的位置
        'x':0.5,  # 標題在水平方向的位置
        'xanchor': 'center',  # 確保標題的中心在 x 的位置
        'yanchor': 'top'      # 確保標題的底部在 y 的位置
    })

fig.show()